# PyCity Schools Analysis

- Your analysis here
  
---Unfortunately, I can't seem to figure out how to solve this error of having non-numeric data in the math scores and possibly english scores. Really frustrating. I will keep trying and hope to resubmit.

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

# Clean the data by dropping rows with non-numeric math_score or reading_score
school_data_complete = school_data_complete[pd.to_numeric(school_data_complete["math_score"], errors='coerce').notna()]
school_data_complete = school_data_complete[pd.to_numeric(school_data_complete["reading_score"], errors='coerce').notna()]


school_data_complete.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [62]:
# Check for unique values in math_score and reading_score
school_data_complete['math_score'].unique()
school_data_complete['reading_score'].unique()


array([66, 94, 90, 67, 97, 82, 96, 95, 78, 64, 71, 85, 88, 74, 89, 91, 99,
       81, 92, 63, 80, 73, 93, 65, 76, 79, 75, 77, 69, 83, 72, 87, 70, 84,
       86, 68, 98], dtype=int64)

In [2]:
school_data_complete.dtypes


Student ID        int64
student_name     object
gender           object
grade            object
school_name      object
reading_score     int64
math_score        int64
School ID         int64
type             object
size              int64
budget            int64
dtype: object


In [3]:
# Convert math_score and reading_score to numeric, replace non-convertible values with NaN
school_data_complete["math_score"] = pd.to_numeric(school_data_complete["math_score"], errors='coerce')
school_data_complete["reading_score"] = pd.to_numeric(school_data_complete["reading_score"], errors='coerce')

# Drop rows with NaN values in math_score or reading_score
school_data_complete = school_data_complete.dropna(subset=["math_score", "reading_score"])


In [4]:
# Convert math_score and reading_score to float64
school_data_complete["math_score"] = school_data_complete["math_score"].astype(float)
school_data_complete["reading_score"] = school_data_complete["reading_score"].astype(float)

# Check if the conversion was successful
print(school_data_complete.dtypes[["math_score", "reading_score"]])


math_score       float64
reading_score    float64
dtype: object


In [5]:
# Group by school_name and calculate the mean math_score and reading_score
per_school_math = school_data_complete.groupby("school_name")["math_score"].mean()
per_school_reading = school_data_complete.groupby("school_name")["reading_score"].mean()

# Convert to DataFrames for clarity
per_school_math_df = per_school_math.reset_index()
per_school_reading_df = per_school_reading.reset_index()

# Display results to check for accuracy
print(per_school_math_df)
print(per_school_reading_df)

              school_name  math_score
0      Bailey High School   77.048432
1     Cabrera High School   83.061895
2    Figueroa High School   76.711767
3        Ford High School   77.102592
4     Griffin High School   83.351499
5   Hernandez High School   77.289752
6      Holden High School   83.803279
7       Huang High School   76.629414
8     Johnson High School   77.072464
9        Pena High School   83.839917
10  Rodriguez High School   76.842711
11    Shelton High School   83.359455
12     Thomas High School   83.418349
13     Wilson High School   83.274201
14     Wright High School   83.682222
              school_name  reading_score
0      Bailey High School      81.033963
1     Cabrera High School      83.975780
2    Figueroa High School      81.158020
3        Ford High School      80.746258
4     Griffin High School      83.816757
5   Hernandez High School      80.934412
6      Holden High School      83.814988
7       Huang High School      81.182722
8     Johnson High Scho

In [6]:
# Merge the two DataFrames back together
per_school_scores_df = pd.merge(per_school_math_df, per_school_reading_df, on="school_name", suffixes=("_math", "_reading"))

# Display the merged DataFrame
print(per_school_scores_df)


              school_name  math_score  reading_score
0      Bailey High School   77.048432      81.033963
1     Cabrera High School   83.061895      83.975780
2    Figueroa High School   76.711767      81.158020
3        Ford High School   77.102592      80.746258
4     Griffin High School   83.351499      83.816757
5   Hernandez High School   77.289752      80.934412
6      Holden High School   83.803279      83.814988
7       Huang High School   76.629414      81.182722
8     Johnson High School   77.072464      80.966394
9        Pena High School   83.839917      84.044699
10  Rodriguez High School   76.842711      80.744686
11    Shelton High School   83.359455      83.725724
12     Thomas High School   83.418349      83.848930
13     Wilson High School   83.274201      83.989488
14     Wright High School   83.682222      83.955000


## District Summary

In [7]:
# Calculate the total number of unique schools
school_count = school_data_complete["school_name"].nunique()

school_count


15

In [8]:
# Calculate the total number of students
student_count = school_data_complete["Student ID"].count()
student_count


39170

In [9]:
# Calculate the total budget
total_budget = school_data_complete["budget"].sum()
total_budget


82932329558

In [10]:
# Calculate the average (mean) math score
average_math_score = school_data_complete["math_score"].mean() 
average_math_score


78.98537145774827

In [11]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete["reading_score"].mean()
average_reading_score


81.87784018381414

In [12]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage


74.9808526933878

In [13]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [14]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate


65.17232575950983

In [15]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({
    "Total Schools": [school_count],
    "Total Students": [student_count],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "% Passing Math": [passing_math_percentage],
    "% Passing Reading": [passing_reading_percentage],
    "% Overall Passing": [overall_passing_rate]
})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$82,932,329,558.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [16]:
# Use the code provided to select the type per school from school_data
school_types = school_data.set_index(["school_name"])["type"]
school_types

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

In [17]:
# Calculate the total student count per school from school_data
per_school_counts = school_data_complete["school_name"].value_counts()
per_school_counts

school_name
Bailey High School       4976
Johnson High School      4761
Hernandez High School    4635
Rodriguez High School    3999
Figueroa High School     2949
Huang High School        2917
Ford High School         2739
Wilson High School       2283
Cabrera High School      1858
Wright High School       1800
Shelton High School      1761
Thomas High School       1635
Griffin High School      1468
Pena High School          962
Holden High School        427
Name: count, dtype: int64

In [18]:
# Calculate the total school budget and per capita spending per school from school_data
per_school_budget = school_data.set_index(["school_name"])["budget"]
per_school_capita = per_school_budget / per_school_counts
per_school_budget
per_school_capita

school_name
Bailey High School       628.0
Cabrera High School      582.0
Figueroa High School     639.0
Ford High School         644.0
Griffin High School      625.0
Hernandez High School    652.0
Holden High School       581.0
Huang High School        655.0
Johnson High School      650.0
Pena High School         609.0
Rodriguez High School    637.0
Shelton High School      600.0
Thomas High School       638.0
Wilson High School       578.0
Wright High School       583.0
dtype: float64

In [19]:
# Inspect the data types and check for non-numeric values in math_score and reading_score columns
print(school_data_complete.dtypes)
print(school_data_complete["math_score"].unique())
print(school_data_complete["reading_score"].unique())
# Force conversion to numeric, replace non-convertible values with NaN, and drop those rows
school_data_complete["math_score"] = pd.to_numeric(school_data_complete["math_score"], errors='coerce')
school_data_complete["reading_score"] = pd.to_numeric(school_data_complete["reading_score"], errors='coerce')

# Drop rows with NaN values in math_score or reading_score
school_data_complete = school_data_complete.dropna(subset=["math_score", "reading_score"])


Student ID         int64
student_name      object
gender            object
grade             object
school_name       object
reading_score    float64
math_score       float64
School ID          int64
type              object
size               int64
budget             int64
dtype: object
[79. 61. 60. 58. 84. 94. 80. 69. 87. 70. 77. 67. 88. 55. 75. 89. 68. 73.
 85. 99. 65. 90. 74. 76. 66. 95. 93. 59. 81. 56. 78. 71. 72. 57. 82. 62.
 63. 86. 98. 91. 92. 97. 83. 96. 64.]
[66. 94. 90. 67. 97. 82. 96. 95. 78. 64. 71. 85. 88. 74. 89. 91. 99. 81.
 92. 63. 80. 73. 93. 65. 76. 79. 75. 77. 69. 83. 72. 87. 70. 84. 86. 68.
 98.]


In [20]:
# Check for duplicate school names
print(school_data_complete["school_name"].duplicated().sum())

39155


In [21]:
# Simplified groupby operation for math_score
try:
    per_school_math_simple = school_data_complete.groupby("school_name")["math_score"].mean()
    print(per_school_math_simple)
except Exception as e:
    print(f"Error occurred: {e}")

# Simplified groupby operation for reading_score
try:
    per_school_reading_simple = school_data_complete.groupby("school_name")["reading_score"].mean()
    print(per_school_reading_simple)
except Exception as e:
    print(f"Error occurred: {e}")
# Re-read the data to reset any previous changes
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


school_name
Bailey High School       77.048432
Cabrera High School      83.061895
Figueroa High School     76.711767
Ford High School         77.102592
Griffin High School      83.351499
Hernandez High School    77.289752
Holden High School       83.803279
Huang High School        76.629414
Johnson High School      77.072464
Pena High School         83.839917
Rodriguez High School    76.842711
Shelton High School      83.359455
Thomas High School       83.418349
Wilson High School       83.274201
Wright High School       83.682222
Name: math_score, dtype: float64
school_name
Bailey High School       81.033963
Cabrera High School      83.975780
Figueroa High School     81.158020
Ford High School         80.746258
Griffin High School      83.816757
Hernandez High School    80.934412
Holden High School       83.814988
Huang High School        81.182722
Johnson High School      80.966394
Pena High School         84.044699
Rodriguez High School    80.744686
Shelton High School      83.72572

In [22]:
try:
    groupby_test = school_data_complete.groupby("school_name").first()
    print(groupby_test)
except Exception as e:
    print(f"Error occurred: {e}")


                       Student ID      student_name gender grade  \
school_name                                                        
Bailey High School          17871      Blake Martin      M   9th   
Cabrera High School         16013      Olivia Short      F  11th   
Figueroa High School         2917        Amy Jacobs      F  10th   
Ford High School            34796   Michael Mercado      M   9th   
Griffin High School         12262    Heather Wright      F  11th   
Hernandez High School        7627     Russell Davis      M  10th   
Holden High School          22847  Daniel Rodriguez      M  11th   
Huang High School               0      Paul Bradley      M   9th   
Johnson High School         30035        Lisa Casey      F  12th   
Pena High School            23274        Alec Davis      M   9th   
Rodriguez High School       26036    Sherry Jenkins      F  11th   
Shelton High School          5866  Jamie Montgomery      F  12th   
Thomas High School          37535        Norma M

In [23]:
# Calculate the number of students per school with math scores of 70 or higher from school_data_complete
students_passing_math = school_data_complete[school_data_complete["math_score"] >= 70]
school_students_passing_math = students_passing_math.groupby(["school_name"]).count()["student_name"]
students_passing_math
school_students_passing_math

school_name
Bailey High School       3318
Cabrera High School      1749
Figueroa High School     1946
Ford High School         1871
Griffin High School      1371
Hernandez High School    3094
Holden High School        395
Huang High School        1916
Johnson High School      3145
Pena High School          910
Rodriguez High School    2654
Shelton High School      1653
Thomas High School       1525
Wilson High School       2143
Wright High School       1680
Name: student_name, dtype: int64

In [24]:
# Calculate the number of students per school with reading scores of 70 or higher from school_data_complete
students_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70]
school_students_passing_reading = students_passing_reading.groupby(["school_name"]).count()["student_name"]
students_passing_reading
school_students_passing_reading

school_name
Bailey High School       4077
Cabrera High School      1803
Figueroa High School     2381
Ford High School         2172
Griffin High School      1426
Hernandez High School    3748
Holden High School        411
Huang High School        2372
Johnson High School      3867
Pena High School          923
Rodriguez High School    3208
Shelton High School      1688
Thomas High School       1591
Wilson High School       2204
Wright High School       1739
Name: student_name, dtype: int64

In [25]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()
school_students_passing_math_and_reading

school_name
Bailey High School       2719
Cabrera High School      1697
Figueroa High School     1569
Ford High School         1487
Griffin High School      1330
Hernandez High School    2481
Holden High School        381
Huang High School        1561
Johnson High School      2549
Pena High School          871
Rodriguez High School    2119
Shelton High School      1583
Thomas High School       1487
Wilson High School       2068
Wright High School       1626
dtype: int64

In [26]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100
per_school_passing_math
per_school_passing_reading
overall_passing_rate


school_name
Bailey High School       54.642283
Cabrera High School      91.334769
Figueroa High School     53.204476
Ford High School         54.289887
Griffin High School      90.599455
Hernandez High School    53.527508
Holden High School       89.227166
Huang High School        53.513884
Johnson High School      53.539172
Pena High School         90.540541
Rodriguez High School    52.988247
Shelton High School      89.892107
Thomas High School       90.948012
Wilson High School       90.582567
Wright High School       90.333333
dtype: float64

In [27]:
# Calculate the total student count per school from school_data
per_school_counts = school_data_complete["school_name"].value_counts()
per_school_counts

school_name
Bailey High School       4976
Johnson High School      4761
Hernandez High School    4635
Rodriguez High School    3999
Figueroa High School     2949
Huang High School        2917
Ford High School         2739
Wilson High School       2283
Cabrera High School      1858
Wright High School       1800
Shelton High School      1761
Thomas High School       1635
Griffin High School      1468
Pena High School          962
Holden High School        427
Name: count, dtype: int64

In [28]:
# Group by school_name and calculate the mean budget
per_school_budget = school_data_complete.groupby("school_name")["budget"].mean()

# Convert to DataFrame for clarity
per_school_budget_df = per_school_budget.reset_index()

# Display results to check for accuracy
print(per_school_budget_df)

              school_name     budget
0      Bailey High School  3124928.0
1     Cabrera High School  1081356.0
2    Figueroa High School  1884411.0
3        Ford High School  1763916.0
4     Griffin High School   917500.0
5   Hernandez High School  3022020.0
6      Holden High School   248087.0
7       Huang High School  1910635.0
8     Johnson High School  3094650.0
9        Pena High School   585858.0
10  Rodriguez High School  2547363.0
11    Shelton High School  1056600.0
12     Thomas High School  1043130.0
13     Wilson High School  1319574.0
14     Wright High School  1049400.0


In [29]:
# Calculate per school capita
per_school_capita = per_school_budget_df["budget"] / per_school_counts

# Convert to DataFrame for clarity
per_school_capita_df = per_school_capita.reset_index()


# Display the results to check for accuracy
print(per_school_capita_df)


                    index   0
0                       0 NaN
1                       1 NaN
2                       2 NaN
3                       3 NaN
4                       4 NaN
5                       5 NaN
6                       6 NaN
7                       7 NaN
8                       8 NaN
9                       9 NaN
10                     10 NaN
11                     11 NaN
12                     12 NaN
13                     13 NaN
14                     14 NaN
15     Bailey High School NaN
16    Cabrera High School NaN
17   Figueroa High School NaN
18       Ford High School NaN
19    Griffin High School NaN
20  Hernandez High School NaN
21     Holden High School NaN
22      Huang High School NaN
23    Johnson High School NaN
24       Pena High School NaN
25  Rodriguez High School NaN
26    Shelton High School NaN
27     Thomas High School NaN
28     Wilson High School NaN
29     Wright High School NaN


In [30]:
# Calculate the percentage of students with a passing math score
per_school_passing_math = school_data_complete[(school_data_complete["math_score"] >= 70)].groupby(["school_name"]).count()["Student ID"] / per_school_counts * 100

# Calculate the percentage of students with a passing reading score
per_school_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)].groupby(["school_name"]).count()["Student ID"] / per_school_counts * 100

# Calculate the overall passing rate (average of the above two)
per_school_overall_passing = (per_school_passing_math + per_school_passing_reading) / 2


In [31]:
# Convert all relevant columns to numeric, forcing any non-numeric values to NaN and then dropping them
per_school_passing_math = pd.to_numeric(per_school_passing_math, errors='coerce')
per_school_passing_reading = pd.to_numeric(per_school_passing_reading, errors='coerce')
per_school_overall_passing = pd.to_numeric(per_school_overall_passing, errors='coerce')

# Recheck the types to make sure they are now consistent
print(per_school_passing_math.dtype)
print(per_school_passing_reading.dtype)
print(per_school_overall_passing.dtype)


float64
float64
float64


In [32]:
print(per_school_counts.dtype)
print(per_school_budget.dtype)
print(per_school_capita.dtype)
print(per_school_math.dtype)
print(per_school_reading.dtype)
print(per_school_passing_math.dtype)
print(per_school_passing_reading.dtype)
print(per_school_overall_passing.dtype)


int64
float64
float64
float64
float64
float64
float64
float64


In [33]:
per_school_counts = pd.to_numeric(per_school_counts, errors='coerce')
per_school_budget = pd.to_numeric(per_school_budget, errors='coerce')
per_school_capita = pd.to_numeric(per_school_capita, errors='coerce')
per_school_math = pd.to_numeric(per_school_math, errors='coerce')
per_school_reading = pd.to_numeric(per_school_reading, errors='coerce')
per_school_passing_math = pd.to_numeric(per_school_passing_math, errors='coerce')
per_school_passing_reading = pd.to_numeric(per_school_passing_reading, errors='coerce')
per_school_overall_passing = pd.to_numeric(per_school_overall_passing, errors='coerce')


In [34]:
per_school_counts = per_school_counts.fillna(0)
per_school_budget = per_school_budget.fillna(0)
per_school_capita = per_school_capita.fillna(0)
per_school_math = per_school_math.fillna(0)
per_school_reading = per_school_reading.fillna(0)
per_school_passing_math = per_school_passing_math.fillna(0)
per_school_passing_reading = per_school_passing_reading.fillna(0)
per_school_overall_passing = per_school_overall_passing.fillna(0)


In [35]:
print(per_school_counts.index)
print(per_school_budget.index)
print(per_school_capita.index)
print(per_school_math.index)
print(per_school_reading.index)
print(per_school_passing_math.index)
print(per_school_passing_reading.index)
print(per_school_overall_passing.index)


Index(['Bailey High School', 'Johnson High School', 'Hernandez High School',
       'Rodriguez High School', 'Figueroa High School', 'Huang High School',
       'Ford High School', 'Wilson High School', 'Cabrera High School',
       'Wright High School', 'Shelton High School', 'Thomas High School',
       'Griffin High School', 'Pena High School', 'Holden High School'],
      dtype='object', name='school_name')
Index(['Bailey High School', 'Cabrera High School', 'Figueroa High School',
       'Ford High School', 'Griffin High School', 'Hernandez High School',
       'Holden High School', 'Huang High School', 'Johnson High School',
       'Pena High School', 'Rodriguez High School', 'Shelton High School',
       'Thomas High School', 'Wilson High School', 'Wright High School'],
      dtype='object', name='school_name')
Index([                      0,                       1,
                             2,                       3,
                             4,                       5,

In [36]:
common_index = per_school_counts.index

per_school_budget = per_school_budget.reindex(common_index)
per_school_capita = per_school_capita.reindex(common_index)
per_school_math = per_school_math.reindex(common_index)
per_school_reading = per_school_reading.reindex(common_index)
per_school_passing_math = per_school_passing_math.reindex(common_index)
per_school_passing_reading = per_school_passing_reading.reindex(common_index)
per_school_overall_passing = per_school_overall_passing.reindex(common_index)


In [37]:
per_school_summary = pd.DataFrame({
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": per_school_overall_passing
})

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary



,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,
Bailey High School,4976,"$3,124,928.00",$0.00,77.048432,81.033963,66.680064,81.933280,74.306672
Johnson High School,4761,"$3,094,650.00",$0.00,77.072464,80.966394,66.057551,81.222432,73.639992
Hernandez High School,4635,"$3,022,020.00",$0.00,77.289752,80.934412,66.752967,80.862999,73.807983
Rodriguez High School,3999,"$2,547,363.00",$0.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,2949,"$1,884,411.00",$0.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,2917,"$1,910,635.00",$0.00,76.629414,81.182722,65.683922,81.316421,73.500171
Ford High School,2739,"$1,763,916.00",$0.00,77.102592,80.746258,68.309602,79.299014,73.804308
Wilson High School,2283,"$1,319,574.00",$0.00,83.274201,83.989488,93.867718,96.539641,95.203679
Cabrera High School,1858,"$1,081,356.00",$0.00,83.061895,83.975780,94.133477,97.039828,95.586652


In [100]:
# Calculate the Per Student Budget
per_school_summary["Per Student Budget"] = per_school_summary["Total School Budget"] / per_school_summary["Total Students"]

# Inspect the first few rows to verify the calculation
print(per_school_summary[["Total School Budget", "Total Students", "Per Student Budget"]].head(20))

# Create a copy of the school_summary since it has the "Per Student Budget" column
school_spending_df = per_school_summary.copy()


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [99]:
# Inspect the original values in the "Per Student Budget" column before conversion and cleaning
print(school_spending_df["Per Student Budget"].head(20))  # Display the first 20 values to get an idea


school_name
Bailey High School       $0.00
Johnson High School      $0.00
Hernandez High School    $0.00
Rodriguez High School    $0.00
Figueroa High School     $0.00
Huang High School        $0.00
Ford High School         $0.00
Wilson High School       $0.00
Cabrera High School      $0.00
Wright High School       $0.00
Shelton High School      $0.00
Thomas High School       $0.00
Griffin High School      $0.00
Pena High School         $0.00
Holden High School       $0.00
Name: Per Student Budget, dtype: object


## Highest-Performing Schools (by % Overall Passing)

In [38]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(["% Overall Passing"], ascending=False)
top_schools.head(5)


,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,
Cabrera High School,1858,"$1,081,356.00",$0.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,1635,"$1,043,130.00",$0.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,962,"$585,858.00",$0.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,1468,"$917,500.00",$0.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,2283,"$1,319,574.00",$0.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By % Overall Passing)

In [39]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(["% Overall Passing"], ascending=True)
bottom_schools.head(5)


,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,
Rodriguez High School,3999,"$2,547,363.00",$0.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,2949,"$1,884,411.00",$0.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,2917,"$1,910,635.00",$0.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,4761,"$3,094,650.00",$0.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,2739,"$1,763,916.00",$0.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

In [43]:
school_data_complete.dtypes
school_data_complete["math_score"].unique()


array([79, 61, 60, 58, 84, 94, 80, 69, 87, 70, 77, 67, 88, 55, 75, 89, 68,
       73, 85, 99, 65, 90, 74, 76, 66, 95, 93, 59, 81, 56, 78, 71, 72, 57,
       82, 62, 63, 86, 98, 91, 92, 97, 83, 96, 64], dtype=int64)

In [42]:
school_data_complete["math_score"] = pd.to_numeric(school_data_complete["math_score"], errors='coerce')


In [44]:
school_data_complete = school_data_complete.dropna(subset=["math_score"])


In [45]:
school_data_complete = school_data_complete.dropna(subset=["math_score"])


In [63]:
# Check for unique values in math_score, grade, and school_name
print(school_data_complete["math_score"].unique())
print(school_data_complete["grade"].unique())
print(school_data_complete["school_name"].unique())


[79 61 60 58 84 94 80 69 87 70 77 67 88 55 75 89 68 73 85 99 65 90 74 76
 66 95 93 59 81 56 78 71 72 57 82 62 63 86 98 91 92 97 83 96 64]
['9th' '12th' '11th' '10th']
['Huang High School' 'Figueroa High School' 'Shelton High School'
 'Hernandez High School' 'Griffin High School' 'Wilson High School'
 'Cabrera High School' 'Bailey High School' 'Holden High School'
 'Pena High School' 'Wright High School' 'Rodriguez High School'
 'Johnson High School' 'Ford High School' 'Thomas High School']


In [64]:
# Group by `school_name` and `grade` to calculate the mean `math_score` for each grade in each school.
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

In [65]:
# Inspect the data types and unique values in math_score
ninth_graders["math_score"].dtypes
ninth_graders["math_score"].unique()
tenth_graders["math_score"].dtypes
tenth_graders["math_score"].unique()
eleventh_graders["math_score"].dtypes
eleventh_graders["math_score"].unique()
twelfth_graders["math_score"].dtypes
twelfth_graders["math_score"].unique()

array([61, 60, 58, 69, 88, 68, 77, 59, 78, 71, 57, 82, 65, 76, 73, 67, 74,
       86, 81, 66, 79, 92, 91, 98, 72, 63, 90, 56, 85, 97, 83, 75, 89, 55,
       87, 94, 93, 95, 80, 96, 84, 64, 62, 70, 99], dtype=int64)

In [67]:
# Group by `school_name` and `grade` to prepare for calculations
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]


In [68]:
# Calculate the mean math scores for each grade
ninth_grade_math_scores = ninth_graders.groupby(["school_name"])["math_score"].mean()
tenth_grade_math_scores = tenth_graders.groupby(["school_name"])["math_score"].mean()
eleventh_grade_math_scores = eleventh_graders.groupby(["school_name"])["math_score"].mean()
twelfth_grade_math_scores = twelfth_graders.groupby(["school_name"])["math_score"].mean()


In [69]:
# Reset the index for clarity
ninth_grade_math_scores_df = ninth_grade_math_scores.reset_index()
tenth_grade_math_scores_df = tenth_grade_math_scores.reset_index()
eleventh_grade_math_scores_df = eleventh_grade_math_scores.reset_index()
twelfth_grade_math_scores_df = twelfth_grade_math_scores.reset_index()


In [70]:
# Merge the DataFrames for each grade
math_scores_by_grade = pd.merge(ninth_grade_math_scores_df, tenth_grade_math_scores_df, on="school_name", suffixes=("_9th", "_10th"))
math_scores_by_grade = pd.merge(math_scores_by_grade, eleventh_grade_math_scores_df, on="school_name")
math_scores_by_grade = pd.merge(math_scores_by_grade, twelfth_grade_math_scores_df, on="school_name", suffixes=("_11th", "_12th"))

# Rename columns for clarity
math_scores_by_grade.columns = ["School Name", "9th", "10th", "11th", "12th"]

# Display the final DataFrame
print(math_scores_by_grade)


              School Name        9th       10th       11th       12th
0      Bailey High School  77.083676  76.996772  77.515588  76.492218
1     Cabrera High School  83.094697  83.154506  82.765560  83.277487
2    Figueroa High School  76.403037  76.539974  76.884344  77.151369
3        Ford High School  77.361345  77.672316  76.918058  76.179963
4     Griffin High School  82.044010  84.229064  83.842105  83.356164
5   Hernandez High School  77.438495  77.337408  77.136029  77.186567
6      Holden High School  83.787402  83.429825  85.000000  82.855422
7       Huang High School  77.027251  75.908735  76.446602  77.225641
8     Johnson High School  77.187857  76.691117  77.491653  76.863248
9        Pena High School  83.625455  83.372000  84.328125  84.121547
10  Rodriguez High School  76.859966  76.612500  76.395626  77.690748
11    Shelton High School  83.420755  82.917411  83.383495  83.778976
12     Thomas High School  83.590022  83.087886  83.498795  83.497041
13     Wilson High S

## Reading Score by Grade 

In [71]:
# Group by `school_name` and `grade` to prepare for calculations
ninth_graders_reading = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders_reading = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders_reading = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders_reading = school_data_complete[(school_data_complete["grade"] == "12th")]


In [72]:
# Calculate the mean reading scores for each grade
ninth_grade_reading_scores = ninth_graders_reading.groupby(["school_name"])["reading_score"].mean()
tenth_grade_reading_scores = tenth_graders_reading.groupby(["school_name"])["reading_score"].mean()
eleventh_grade_reading_scores = eleventh_graders_reading.groupby(["school_name"])["reading_score"].mean()
twelfth_grade_reading_scores = twelfth_graders_reading.groupby(["school_name"])["reading_score"].mean()


In [73]:
# Reset the index for clarity
ninth_grade_reading_scores_df = ninth_grade_reading_scores.reset_index()
tenth_grade_reading_scores_df = tenth_grade_reading_scores.reset_index()
eleventh_grade_reading_scores_df = eleventh_grade_reading_scores.reset_index()
twelfth_grade_reading_scores_df = twelfth_grade_reading_scores.reset_index()


In [74]:
# Merge the DataFrames for each grade
reading_scores_by_grade = pd.merge(ninth_grade_reading_scores_df, tenth_grade_reading_scores_df, on="school_name", suffixes=("_9th", "_10th"))
reading_scores_by_grade = pd.merge(reading_scores_by_grade, eleventh_grade_reading_scores_df, on="school_name")
reading_scores_by_grade = pd.merge(reading_scores_by_grade, twelfth_grade_reading_scores_df, on="school_name", suffixes=("_11th", "_12th"))

# Rename columns for clarity
reading_scores_by_grade.columns = ["School Name", "9th", "10th", "11th", "12th"]

# Display the final DataFrame
print(reading_scores_by_grade)


              School Name        9th       10th       11th       12th
0      Bailey High School  81.303155  80.907183  80.945643  80.912451
1     Cabrera High School  83.676136  84.253219  83.788382  84.287958
2    Figueroa High School  81.198598  81.408912  80.640339  81.384863
3        Ford High School  80.632653  81.262712  80.403642  80.662338
4     Griffin High School  83.369193  83.706897  84.288089  84.013699
5   Hernandez High School  80.866860  80.660147  81.396140  80.857143
6      Holden High School  83.677165  83.324561  83.815534  84.698795
7       Huang High School  81.290284  81.512386  81.417476  80.305983
8     Johnson High School  81.260714  80.773431  80.616027  81.227564
9        Pena High School  83.807273  83.612000  84.335938  84.591160
10  Rodriguez High School  80.993127  80.629808  80.864811  80.376426
11    Shelton High School  84.122642  83.441964  84.373786  82.781671
12     Thomas High School  83.728850  84.254157  83.585542  83.831361
13     Wilson High S

## Scores by School Spending

In [93]:
# Re-inspect the unique values in the "Per Student Budget" column
print(school_spending_df["Per Student Budget"].unique())

# Check for any NaN values in the "Per Student Budget" column
print(school_spending_df["Per Student Budget"].isna().sum())

# Ensure all values in the "Per Student Budget" column are numeric
school_spending_df["Per Student Budget"] = pd.to_numeric(school_spending_df["Per Student Budget"], errors='coerce')

# Drop any rows with NaN values in the "Per Student Budget" column after conversion
school_spending_df = school_spending_df.dropna(subset=["Per Student Budget"])

# Verify that there are no NaN values left
print(school_spending_df["Per Student Budget"].isna().sum())


[]
0
0


In [97]:
# Inspect the values in the "Per Student Budget" column after conversion and dropping NaNs
print(school_spending_df["Per Student Budget"].describe())
print(school_spending_df["Per Student Budget"].unique())


count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: Per Student Budget, dtype: float64
[]


In [94]:
# Establish the bins
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]


In [95]:
# Reattempt the pd.cut operation to create the "Spending Ranges (Per Student)" column
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], bins=spending_bins, labels=labels)

# Check if the column has been created correctly
print(school_spending_df.head())



Empty DataFrame
Columns: [Total Students, Total School Budget, Per Student Budget, Average Math Score, Average Reading Score, % Passing Math, % Passing Reading, % Overall Passing, School Size, Spending Ranges (Per Student)]
Index: []


In [96]:
# Reattempt the groupby operations for spending ranges
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

# Check the results to ensure they are correct
print(spending_math_scores)
print(spending_reading_scores)
print(spending_passing_math)
print(spending_passing_reading)
print(overall_passing_spending)


Spending Ranges (Per Student)
<$585      NaN
$585-630   NaN
$630-645   NaN
$645-680   NaN
Name: Average Math Score, dtype: float64
Spending Ranges (Per Student)
<$585      NaN
$585-630   NaN
$630-645   NaN
$645-680   NaN
Name: Average Reading Score, dtype: float64
Spending Ranges (Per Student)
<$585      NaN
$585-630   NaN
$630-645   NaN
$645-680   NaN
Name: % Passing Math, dtype: float64
Spending Ranges (Per Student)
<$585      NaN
$585-630   NaN
$630-645   NaN
$645-680   NaN
Name: % Passing Reading, dtype: float64
Spending Ranges (Per Student)
<$585      NaN
$585-630   NaN
$630-645   NaN
$645-680   NaN
Name: % Overall Passing, dtype: float64


C:\Users\Hester\AppData\Local\Temp\ipykernel_13168\2252184066.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
C:\Users\Hester\AppData\Local\Temp\ipykernel_13168\2252184066.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
C:\Users\Hester\AppData\Local\Temp\ipykernel_13168\2252184066.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True

In [84]:
# Create a copy of the school summary since it has the "Per Student Budget"
school_spending_df = per_school_summary.copy()


In [85]:
# Inspect the unique values in the "Per Student Budget" column
print(school_spending_df["Per Student Budget"].unique())


['$0.00']


In [86]:
# Convert "Per Student Budget" to numeric, forcing errors to NaN
school_spending_df["Per Student Budget"] = pd.to_numeric(school_spending_df["Per Student Budget"], errors='coerce')

# Drop rows with NaN values in "Per Student Budget"
school_spending_df = school_spending_df.dropna(subset=["Per Student Budget"])


In [87]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], spending_bins, labels=labels)


In [88]:
# Check if "Spending Ranges (Per Student)" column exists and inspect its first few entries
print(school_spending_df.head())


Empty DataFrame
Columns: [Total Students, Total School Budget, Per Student Budget, Average Math Score, Average Reading Score, % Passing Math, % Passing Reading, % Overall Passing, School Size, Spending Ranges (Per Student)]
Index: []


In [89]:
# Inspect the unique values in the "Per Student Budget" column again
print(school_spending_df["Per Student Budget"].unique())

# Check the bins and labels to ensure they are set up correctly
print(spending_bins)
print(labels)


[]
[0, 585, 630, 645, 680]
['<$585', '$585-630', '$630-645', '$645-680']


In [90]:
# Reattempt the pd.cut operation
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], bins=spending_bins, labels=labels)

# Check if the column has been created correctly
print(school_spending_df.head())


Empty DataFrame
Columns: [Total Students, Total School Budget, Per Student Budget, Average Math Score, Average Reading Score, % Passing Math, % Passing Reading, % Overall Passing, School Size, Spending Ranges (Per Student)]
Index: []


In [92]:
# Reattempt the groupby operations
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

# Check the results to ensure they are correct
print(spending_math_scores)
print(spending_reading_scores)
print(spending_passing_math)
print(spending_passing_reading)
print(overall_passing_spending)



Spending Ranges (Per Student)
<$585      NaN
$585-630   NaN
$630-645   NaN
$645-680   NaN
Name: Average Math Score, dtype: float64
Spending Ranges (Per Student)
<$585      NaN
$585-630   NaN
$630-645   NaN
$645-680   NaN
Name: Average Reading Score, dtype: float64
Spending Ranges (Per Student)
<$585      NaN
$585-630   NaN
$630-645   NaN
$645-680   NaN
Name: % Passing Math, dtype: float64
Spending Ranges (Per Student)
<$585      NaN
$585-630   NaN
$630-645   NaN
$645-680   NaN
Name: % Passing Reading, dtype: float64
Spending Ranges (Per Student)
<$585      NaN
$585-630   NaN
$630-645   NaN
$645-680   NaN
Name: % Overall Passing, dtype: float64


C:\Users\Hester\AppData\Local\Temp\ipykernel_13168\3308965380.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
C:\Users\Hester\AppData\Local\Temp\ipykernel_13168\3308965380.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
C:\Users\Hester\AppData\Local\Temp\ipykernel_13168\3308965380.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True

In [55]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({
    "Average Math Score": spending_math_scores,
    "Average Reading Score": spending_reading_scores,
    "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending
})

# Display results
spending_summary


NameError: name 'spending_math_scores' is not defined

## Scores by School Size

In [56]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


In [57]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], size_bins, labels=labels)
per_school_summary


,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
school_name,,,,,,,,,
Bailey High School,4976,"$3,124,928.00",$0.00,77.048432,81.033963,66.680064,81.933280,74.306672,Large (2000-5000)
Johnson High School,4761,"$3,094,650.00",$0.00,77.072464,80.966394,66.057551,81.222432,73.639992,Large (2000-5000)
Hernandez High School,4635,"$3,022,020.00",$0.00,77.289752,80.934412,66.752967,80.862999,73.807983,Large (2000-5000)
Rodriguez High School,3999,"$2,547,363.00",$0.00,76.842711,80.744686,66.366592,80.220055,73.293323,Large (2000-5000)
Figueroa High School,2949,"$1,884,411.00",$0.00,76.711767,81.158020,65.988471,80.739234,73.363852,Large (2000-5000)
Huang High School,2917,"$1,910,635.00",$0.00,76.629414,81.182722,65.683922,81.316421,73.500171,Large (2000-5000)
Ford High School,2739,"$1,763,916.00",$0.00,77.102592,80.746258,68.309602,79.299014,73.804308,Large (2000-5000)
Wilson High School,2283,"$1,319,574.00",$0.00,83.274201,83.989488,93.867718,96.539641,95.203679,Large (2000-5000)
Cabrera High School,1858,"$1,081,356.00",$0.00,83.061895,83.975780,94.133477,97.039828,95.586652,Medium (1000-2000)


In [58]:
# Calculate averages for the desired columns.
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()


C:\Users\Hester\AppData\Local\Temp\ipykernel_13168\3588142246.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
C:\Users\Hester\AppData\Local\Temp\ipykernel_13168\3588142246.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
C:\Users\Hester\AppData\Local\Temp\ipykernel_13168\3588142246.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observe

In [59]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({
    "Average Math Score": size_math_scores,
    "Average Reading Score": size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing": size_overall_passing
})

# Display results
size_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

In [60]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()


KeyError: 'School Type'

In [61]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({
    "Average Math Score": average_math_score_by_type,
    "Average Reading Score": average_reading_score_by_type,
    "% Passing Math": average_percent_passing_math_by_type,
    "% Passing Reading": average_percent_passing_reading_by_type,
    "% Overall Passing": average_percent_overall_passing_by_type
})
# Display results
type_summary


NameError: name 'average_math_score_by_type' is not defined